In [1]:
from os.path import (
    abspath,
    dirname,
    exists,
    join,
)
from pandas import (
    read_csv,
    merge,
    concat,
    DataFrame,
    Series,
)
from random import randint
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_absolute_percentage_error,
    average_precision_score,
    mean_absolute_error,
    coverage_error,
    ndcg_score,
)

In [2]:
PWD = (abspath(''))
datadir = join(PWD, '../Datasets/ml-1m/')
files = {}
files['train'] = join(datadir, 'train.csv')
files['test'] = join(datadir, 'test.csv')
THRESHOLD = 10
TAKE_TOP = 20

In [3]:
dfs = {}
for file in files:
    dfs[file] = read_csv(
        files[file],
        engine='python',
        encoding='latin1',
    )
    dfs[file]['rating'] = dfs[file]['rating'].astype(int)

In [4]:
dfs['train'].head()

,title,rating,userId
0,Airplane! (1980),4,1
1,Aladdin (1992),4,1
2,Antz (1998),4,1
3,Apollo 13 (1995),5,1
4,Awakenings (1990),5,1


In [5]:
ratings = {}
users = dfs['train']['userId'].to_numpy()
users = set(users)
users = list(users)
for user in users:
    ratings[user] = {}

In [6]:
rows = dfs['train'].to_numpy()
for row in rows:
    ratings[row[2]][row[0]] = row[1]

In [7]:
dfs['test'].head()

,title,rating,userId
0,Snow White and the Seven Dwarfs (1937),4,1
1,"Sound of Music, The (1965)",5,1
2,Star Wars: Episode IV - A New Hope (1977),4,1
3,Tarzan (1999),3,1
4,Titanic (1997),4,1


In [8]:
dfs['train'].shape, dfs['test'].shape

((755865, 3), (99865, 3))

In [9]:
dfs['test'][dfs['test']['title'].isin(dfs['train']['title'])].shape

(99865, 3)

In [10]:
dfs['users'] = dfs['train'].pivot_table(
    index=['title'],
    columns=['userId'],
    values='rating',
)

In [11]:
dfs['users'].head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10 Things I Hate About You (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
101 Dalmatians (1961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0


In [12]:
corr_user = dfs['users'].corr(min_periods=THRESHOLD)

In [13]:
corr_user += 1.0
corr_user /= 2.0

In [14]:
corr_user.head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.656670,0.424752,...,NaN,NaN,NaN,NaN,NaN,0.426223,NaN,NaN,0.5508,0.527408
2,NaN,1.00000,NaN,NaN,0.30215,NaN,0.597590,0.524901,0.602029,0.450896,...,NaN,NaN,0.646647,NaN,0.740294,0.667588,0.646267,NaN,NaN,0.515827
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.510779,...,NaN,NaN,NaN,NaN,NaN,0.610754,NaN,NaN,NaN,0.284105
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.336337,NaN,NaN,0.790146,NaN,NaN,NaN,NaN
5,NaN,0.30215,NaN,NaN,1.00000,NaN,0.656348,0.741886,0.573793,0.464953,...,NaN,NaN,NaN,NaN,0.758365,0.610301,0.445316,NaN,NaN,0.677139


In [15]:
users = corr_user.columns.tolist()

In [16]:
similarities = {}
for u in users:
    users_sim = corr_user[u].dropna()
    similarities[u] = {}
    for (v, s) in zip(users_sim.index, users_sim.values):
        similarities[u][v] = s

In [17]:
dfs['pivot'] = dfs['train'].pivot_table(
    index=['title'],
    columns=['userId'],
    values='rating',
)

In [18]:
def get_harmonic(W, X):
    den = 0
    for (w, x) in zip(W, X):
        den += (w/x)
    num = sum(W)
    res = num/den
    
    return res

In [19]:
%time
rows = dfs['test'].to_numpy()
predictions = []
for row in rows[:]:
    rating = 0
    u = row[2]
    m = row[0]
    r = row[1]
    tot = 0
    raters = dfs['pivot'].loc[m].dropna().index
    sim_tot = 0
    #print(raters)
    W = []
    X = []
    for v in raters:
        if v not in similarities[u]:
            continue
        if similarities[u][v] >= .7:
            rating += ratings[v][m]*similarities[u][v]
            W.append(similarities[u][v])
            X.append(ratings[v][m])
            sim_tot += similarities[u][v]
            tot += 1
            if tot > TAKE_TOP:
                break
    if tot > 0:
        rating /= sim_tot
        rating = get_harmonic(W, X)
        #rating *= 5
        #rating /= 5
    else:
        rating = dfs['pivot'].loc[m].dropna().values
        if len(rating) < 1:
            rating = dfs['user'].loc[u].dropna().values
            rating = sum(rating)/len(rating)
        else:
            rating = sum(rating)/len(rating)
    rating += .5
    rating = int(rating)
    predictions.append(rating)

Wall time: 0 ns


In [20]:
dfs['test']['prediction'] = predictions

In [21]:
dfs['test'].head()

,title,rating,userId,prediction
0,Snow White and the Seven Dwarfs (1937),4,1,4
1,"Sound of Music, The (1965)",5,1,4
2,Star Wars: Episode IV - A New Hope (1977),4,1,5
3,Tarzan (1999),3,1,4
4,Titanic (1997),4,1,4


In [22]:
mean_absolute_percentage_error(dfs['test']['rating'], dfs['test']['prediction'])

0.3169690749178058

In [23]:
mean_absolute_error(dfs['test']['rating'], dfs['test']['prediction'])

0.9064537125118911

In [24]:
def get_map(y_true, y_pred, k=3):
    y_true = [1 if y >= k else 0 for y in y_true]
    y_pred = [1 if y >= k else 0 for y in y_pred]
    
    return average_precision_score(y_true, y_pred)

In [25]:
get_map(dfs['test']['rating'], dfs['test']['prediction'])

0.8931766552067855

In [26]:
get_map(dfs['test']['rating'], dfs['test']['prediction'], 4)

0.6998344504205387

In [27]:
dfs['test']['prediction'].describe()

count    99865.000000
mean         3.652381
std          0.963700
min          1.000000
25%          3.000000
50%          4.000000
75%          4.000000
max          5.000000
Name: prediction, dtype: float64